In [1]:
import pandas as pd
import numpy  as np

In [2]:
# обработка огромного ДФ, удаляем дубликаты по чанкам, чтобы ДФ поместился в оперативку
# df_dup = []
#
# for chunk in pd.read_csv('denis_2.2.csv', sep=';', chunksize=1000000):
#     df_nodup = chunk.drop_duplicates()
#     df_dup.append(df_nodup)
#    
# df = pd.concat(df_dup)   
# df_nodup_final = df.drop_duplicates()
# df_nodup_final.to_csv (r'C:\Users\absolutely alone\Desktop\noduplicates.csv', index=False)

In [4]:
df = pd.read_csv('noduplicates.csv')
df['op_date'] = pd.to_datetime(df['op_date'])
df["year_month"] = df["op_date"].dt.to_period("M")

In [5]:
# текущая дата
current_date   = pd.Timestamp('2023-02-20')
# текущий месяц
current_month  = current_date.to_period('M')
# прошлый месяц
previous_month = current_month - 1
# клиент квалифицирован, если GGR > N (N по умолчанию 500)
N = 500

In [6]:
# датафрейм с данными прошлого месяца
df_previous_month = df[df['year_month'] == previous_month]

In [7]:
# функция, которая считает GGR по каждому пользователю => возвращает user_id и GGR
def GGR(df):
    GGR = pd.pivot_table(df, values='amount', index='user_id', columns=["type", "direction"], aggfunc='sum', fill_value=0).reset_index()
    GGR['GGR'] = GGR['bet']['out'] - GGR['bet']['in']
    return GGR.set_index('user_id')['GGR'].reset_index()
# функция, которая возвращает user_id и наличие 1-го или более депозитов (1 - есть депозит, 0 - нет)
def deposit(df):
    deposit_users = df[df['type'] == 'deposit']['user_id']
    result = df[['user_id']].drop_duplicates().copy()
    result['Сделал 1 и более депозит'] = result['user_id'].isin(deposit_users).map({True: 1, False: 0})
    return result

In [8]:
# пользователи, выполнившие квалификацию в прошлом месяце 
q_users_prev_month = GGR(df_previous_month).query(f'GGR > {N}')
# датафрейм текущего месяца из квалифицированных пользователей прошлого месяца
df_current_month = df[(df['user_id'].isin(q_users_prev_month['user_id'])) & (df['year_month'] == current_month)]

In [9]:
# функция на каждый день текущего месяца до выбранной даты возвращает GGR и наличие депизита по пользователю
def get_stats(current_month_df, end_date):
    result = []
    for day in pd.date_range(end_date.replace(day=1), end_date):
        before_day = current_month_df[current_month_df['op_date'].dt.date <= day.date()]  # транзакции по текущему месяцу от начала месяца до выбранной даты
        day_df = pd.merge(GGR(before_day), deposit(before_day), on='user_id')  # показатели каждого клиента на день
        day_df['Дата текущего месяца'] = day
        result.append(day_df)
    return pd.concat(result)

# получаем данные текущего месяца, и смотрим выполнил ли пользователь квалификацию
current_month_stats = get_stats(df_current_month, current_date)
current_month_stats['Выполнил квалификацию'] = (current_month_stats['GGR'] > N).map({True: 1, False: 0})

# добавляем статические данные (параметры исследования), отбрасываем ненужные поля
df_detalisation = current_month_stats.drop(columns='GGR').assign(**{'Месяц квалификации': previous_month, 'Скор квалификации': N})
df_detalisation = df_detalisation[['Месяц квалификации', 'Скор квалификации', 'user_id', 'Дата текущего месяца', 'Выполнил квалификацию', 'Сделал 1 и более депозит']]

aggregated = df_detalisation.groupby('Дата текущего месяца').sum().reset_index()
df_aggregation = pd.DataFrame({
    'Месяц квалификации':             previous_month, 
    'Количичество квалифицированных': q_users_prev_month['user_id'].nunique(),
    'Скор квалификации':              N,
    'Дата текущего месяца':           aggregated['Дата текущего месяца'],
    'Выполнили квалификацию (RR)':    aggregated['Выполнил квалификацию'] / q_users_prev_month['user_id'].nunique() * 100,
    'Сделали 1 и более депозит (CR)': aggregated['Сделал 1 и более депозит'] / q_users_prev_month['user_id'].nunique() * 100,
})

C:\Users\absolutely alone\AppData\Local\Temp\ipykernel_12576\1677864749.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated = df_detalisation.groupby('Дата текущего месяца').sum().reset_index()


In [10]:
df_detalisation

,Месяц квалификации,Скор квалификации,user_id,Дата текущего месяца,Выполнил квалификацию,Сделал 1 и более депозит
0,2023-01,500,L-102503_993-3,2023-02-01,0,0
1,2023-01,500,L-103751_993-1,2023-02-01,0,0
2,2023-01,500,L-104112_993-2,2023-02-01,1,0
3,2023-01,500,L-105517_993-7,2023-02-01,0,0
4,2023-01,500,L-105531_993-1,2023-02-01,0,1
...,...,...,...,...,...,...
6906,2023-01,500,L-99639_993-9,2023-02-20,0,1
6907,2023-01,500,L-99673_993-3,2023-02-20,0,0
6908,2023-01,500,L-99860_993-0,2023-02-20,0,1
6909,2023-01,500,L-99894_993-4,2023-02-20,0,0


In [11]:
df_aggregation

,Месяц квалификации,Количичество квалифицированных,Скор квалификации,Дата текущего месяца,Выполнили квалификацию (RR),Сделали 1 и более депозит (CR)
0,2023-01,20534,500,2023-02-01,0.272718,1.047044
1,2023-01,20534,500,2023-02-02,0.525957,2.108698
2,2023-01,20534,500,2023-02-03,0.813285,3.097302
3,2023-01,20534,500,2023-02-04,1.061654,4.037207
4,2023-01,20534,500,2023-02-05,1.310022,5.050161
5,2023-01,20534,500,2023-02-06,1.597351,6.189734
6,2023-01,20534,500,2023-02-07,1.797020,7.202688
7,2023-01,20534,500,2023-02-08,2.035648,8.064673
8,2023-01,20534,500,2023-02-09,2.293757,9.043538
9,2023-01,20534,500,2023-02-10,2.600565,10.032142
